In [1]:
import pyodbc
from datetime import date, datetime, timedelta
import pandas as pd
import win32com.client as win32
from databases import Bawm, Crm,PosicaoDm1, BDS, CVM,Boletador
from funcoes_datas import FuncoesDatas
import os
import numpy as np
import xlwings as xw
import unicodedata
from itertools import zip_longest
from emailer import Email 

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation
[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Chamando as classesdo DataBases
posicao = PosicaoDm1()
crm= Crm()
fdt = FuncoesDatas()
bds = BDS()
fdt = FuncoesDatas()
hoje = fdt.hoje()
bol = Boletador()
bawm = Bawm()

In [3]:
def moeda(my_value):
    a = '{:,.2f}'.format(float(my_value))
    b = a.replace(',','v')
    c = b.replace('.',',')
    return c.replace('v','.')

In [4]:
def remove_acentos(texto):
     return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')

In [5]:
#Função que analisa o emissor
def analise_emissor(ativos):
    ativos_bancos = ['lca','cdb','lci','lig','lf','dpge','lfs']
    coe = ['coe']
    bancos_aprovados = ['btg','brad','sant','bradesco','rabobank','santander','itaú','banco do brasil','bb','cef','caixa economica','bnds','nu','mercedez','bmg','bnp','agricole','safra','bbm','daycoval','votorantim','cnh','itaú','alfa','renault','volkswagen','itau','bv']
    ativos = ativos.split()
    if any(ativo in ativos for ativo in ativos_bancos):
        if any(ativo in ativos for ativo in bancos_aprovados):
            banco = 'Recomendado'
        else:
            banco = 'Reprovado'
            
    elif any(ativo in ativos for ativo in coe):
        banco = 'Reprovado'        
    else:
        banco = np.nan
    return banco   

In [6]:
#Função que verifica se o produto é aprovado

def produto_aprovado(cod):
    cod = str(cod)
    

In [7]:
#Posicao em d1
carteira_atual = posicao.posicao_total_depara(hoje)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_atual=carteira_atual.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_atual['GuidProduto']= carteira_atual['GuidProduto'].str.lower()

#Retirar da carteira Caixa e Compromissada
carteira_atual = carteira_atual[(carteira_atual['TipoProduto']!='CAIXA')&(carteira_atual['TipoProduto']!='COMP')]
carteira_atual['NomeContaCRM'] = carteira_atual['NomeContaCRM'].apply(lambda x: remove_acentos(x) )

In [8]:
#Busca todos os produtos deliberados e configura os campos para os merges seguintes

produtos = crm.rating_produtos()
produtos['GuidProduto']=produtos['GuidProduto'].str.lower()
produtos['CETIP']=produtos['CETIP'].str.lower()
produtos['ISIN']=produtos['ISIN'].str.lower()

In [9]:
produtos.to_excel('produtos.xlsx')

In [10]:
#Produtos aprovados pela casa

pasta = 'O:/SAO/CICH_All/Real Estate/1. Users/Guilherme Liberatore/2. Real Estate/3. Monitoramento/PDFs'
ativos_imob = os.listdir(pasta)
ativos_imob = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_imob]

pasta = 'O:/SAO/CICH_All/Renda Variável/Monitoramento/Monitoramentos/2024.1T'
ativos_rv = os.listdir(pasta)
ativos_rv  = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_rv]

corporativos =  pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')
corporativos  = corporativos[(corporativos['Status']=='Recomendado')]
corporativos['CETIP'] = corporativos['CETIP'].str.lower()
corporativos = list(corporativos['CETIP'])

estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')
estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')[['IdPAS']].astype('str')
estruturados = estruturados[estruturados['IdPAS'].notnull()].dropna()
estruturados = pd.merge(left=estruturados,right=produtos,on='IdPAS',how='left')

estruturados['CETIP'] = estruturados['CETIP'].str.lower()
estruturados['ISIN'] = estruturados['ISIN'].str.lower()
estruturado_cetip = list(estruturados['CETIP'])
estruturado_isin= list(estruturados['ISIN'])

#Gestores aprovados
gestores = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Gestores')[['Gestora']]
gestores =list(gestores['Gestora'].str.lower())

#Códigos dos ativos aprovados

aprovados = ativos_rv + ativos_imob+corporativos+estruturado_cetip+estruturado_isin
aprovados = [aprovado for aprovado in aprovados if str(aprovado) != 'nan']
aprovados = [aprovado for aprovado in aprovados if aprovado != np.nan]
aprovados = [aprovado for aprovado in aprovados if aprovado != None]

produtos_aprovados = pd.DataFrame(aprovados)

In [11]:
#Verificação dos ativos já deliberados (verificar se o banco é aprovado ou ativo está na lista de crédito)

#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite['Deliberação'] = produtos_comite['Deliberação'].astype('str')
produtos_comite = produtos_comite[produtos_comite['Deliberação'].str.contains('Zerar')]
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite['Cliente'] =produtos_comite['Cliente'].apply(lambda x: remove_acentos(x) )

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Unifica as bases
ativos_deliberados = pd.concat([cnpj,isin,cetip])
ativos_deliberados['Ativo_x'] = ativos_deliberados['Ativo_x'].str.lower()

#Códigos não localizados *** Incluir na planilha de verificação

verificar_ativos = ativos_deliberados[ativos_deliberados['Ativo_y'].isnull()]
ativos_não_localizados = verificar_ativos[['Cliente','Cetip/Isin/CNPJ','Ativo_x']].reset_index(drop=True).drop_duplicates(subset=['Ativo_x']).sort_values(by=['Cliente'])

#Verificar os ativos que estão duplicados*** Incluir na planilha de verificação

ativos_check_duplicados = ativos_deliberados[['Cetip/Isin/CNPJ','Ativo_y','Deliberação']].drop_duplicates()
duplicados = ativos_check_duplicados.groupby(['Cetip/Isin/CNPJ','Ativo_y']).count().reset_index()[['Ativo_y','Deliberação']].astype('str').replace('2','Duplicado').replace('1','ok')
duplicados = pd.merge(left=ativos_check_duplicados,right=duplicados,on='Ativo_y',how='left')
duplicados = duplicados[duplicados['Deliberação_y']=='Duplicado'].sort_values(by=['Ativo_y']).reset_index(drop=True)

#Retorna os ativos com deliberações diferentes e que estão duplicados.
verificar = duplicados[['Cetip/Isin/CNPJ','Ativo_y']].drop_duplicates().rename(columns={'Ativo_y':'Ativo'})
verificar['Verificar'] = 'Duplicidade'


#Verificar deliberação bancos - Bancos aprovados que estão como "zerar"

ativos_deliberados['Ativo_x'] = ativos_deliberados['Ativo_x'].astype('str')
ativos_deliberados['Rating_Banco'] = ativos_deliberados['Ativo_x'].apply(lambda x :analise_emissor(x))
verificar_deliberacao_bancos = ativos_deliberados[ativos_deliberados['Rating_Banco']=='Aprovado'][['Cetip/Isin/CNPJ','Ativo_x','Deliberação','Rating']].rename(columns={'Ativo_x':'Ativo'})
verificar_deliberacao_bancos['Verificar'] = 'Banco Aprovado'

#Atualiza tabela de verificar com os bancos aprovados que estão com a deliberação "zerar" *** Incluir na planilha de verificação
verificar = pd.concat([verificar,verificar_deliberacao_bancos])

#Verifica produtos que estão como recomendado/monitorado no CRM e que estão para serem zerados
verificar_crm = ativos_deliberados[(ativos_deliberados['Rating']=='Recomendado')][['Cetip/Isin/CNPJ','Ativo_x','Deliberação','Rating','CETIP','IdPAS']].rename(columns={'Ativo_x':'Ativo'}).drop_duplicates()[['Cetip/Isin/CNPJ','Ativo','Deliberação','Rating','CETIP','IdPAS']]
verificar_crm['Verificar'] = 'Verificar Cadastro no CRM'
verificar = pd.concat([verificar,verificar_crm])


#Verifica se o ativo é monitorado ou recomendado pela equipe de investimentos e uni a planilha de verificação
lista_ativos_analisar=list(ativos_deliberados['Cetip/Isin/CNPJ'].str.lower())
rating_investimentos = ['Recomendado' if ativo in aprovados else np.nan for ativo in lista_ativos_analisar ]
ativos_deliberados['Rating_Credito/RealEstate'] = rating_investimentos

verifica_ativos_aprov_invest = ativos_deliberados[ativos_deliberados['Rating_Credito/RealEstate'].notnull()]
verifica_ativos_aprov_invest = verifica_ativos_aprov_invest.rename(columns={'Ativo_x':'Ativo'}).drop_duplicates()[['Cetip/Isin/CNPJ','Ativo','Deliberação','Rating','CETIP','IdPAS']]
verifica_ativos_aprov_invest['Verificar'] = 'Ativos Aprovados/Monitorados por investimentos'
verificar = pd.concat([verificar,verifica_ativos_aprov_invest])
verificar = verificar.drop_duplicates(subset=['Ativo'])

#Retirando todos os cod aprovados CRM
lista_ativos_analisar_cetip=list(ativos_deliberados['CETIP'].str.lower())
rating_investimentos_cetip = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in lista_ativos_analisar_cetip ]
ativos_deliberados['CETIP_verificar'] = rating_investimentos_cetip
rating_investimentos_cetip = ativos_deliberados[ativos_deliberados['CETIP_verificar']=='Recomendado'][['Cetip/Isin/CNPJ','Ativo_x','Deliberação','Rating','CETIP','IdPAS']].rename(columns={'Ativo_x':'Ativo'})
rating_investimentos_cetip['Verificar'] = 'Ativos Aprovados/Monitorados por investimentos'
verificar = pd.concat([verificar,rating_investimentos_cetip])

#Retirando todos os cod aprovados CRM
lista_ativos_analisar_ticker=list(ativos_deliberados['IdPAS'].str.lower())
rating_investimentos_ticker = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in lista_ativos_analisar_ticker]
ativos_deliberados['Ticker_verificar'] = rating_investimentos_ticker
rating_investimentos_ticker = ativos_deliberados[ativos_deliberados['Ticker_verificar']=='Recomendado'][['Cetip/Isin/CNPJ','Ativo_x','Deliberação','Rating','CETIP','IdPAS']].rename(columns={'Ativo_x':'Ativo'})
rating_investimentos_ticker['Verificar'] = 'Ativos Aprovados/Monitorados por investimentos'
verificar = pd.concat([verificar,rating_investimentos_ticker]).drop_duplicates().reset_index(drop=True)

#Retornar o rating CRM
verificar = pd.merge(left=verificar,right=produtos, on='IdPAS',how='left')
verificar = verificar[['Cetip/Isin/CNPJ','Ativo_x','Verificar','Deliberação','Rating_y','IdPAS','CETIP_y']].rename(columns={'Ativo_x':'Ativo','Rating_y':'Rating_CRM','IdPAS_y':'IdPAS','CETIP_y':'CETIP'})

#Fazer o merge com os status de crédito
aprovados_credito  =  pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')[['CETIP','Status']]
aprovados_credito ['CETIP'] = aprovados_credito ['CETIP'].str.lower()
aprovados_credito=aprovados_credito[aprovados_credito['CETIP'].notnull()]
fidc_recomendados = estruturados[['CETIP']]
fidc_recomendados['Rating_FICD']='Recomendado'
fidc_recomendados = fidc_recomendados[fidc_recomendados['CETIP'].notnull()]
verificar = pd.merge(left=verificar,right=aprovados_credito, on='CETIP',how='left').rename(columns={'Status':'Rating_Corp.'})
verificar = pd.merge(left=verificar,right=fidc_recomendados, on='CETIP',how='left')
verificar['Rating_Corp.'].loc[verificar['CETIP'].isnull()] = np.nan
verificar['Rating_FICD'].loc[verificar['Verificar'].isnull()] = np.nan
verificar = verificar.drop_duplicates().reset_index(drop=True)

In [12]:
#Retornando os ativos que foram deliberados para venda e possuem posição na D1
ativos_aprovados = list(verificar['Cetip/Isin/CNPJ'])
ativos_deliberados = ativos_deliberados[(ativos_deliberados['Rating']!='Recomendado')|(ativos_deliberados['Rating_Banco']!='Recomendado')]
ativos_deliberados = ativos_deliberados.loc[~ativos_deliberados['Cetip/Isin/CNPJ'].isin(ativos_aprovados)]

ativos_deliberados['GuidProduto'] = ativos_deliberados['GuidProduto'].str.lower()
ativos_deliberados = ativos_deliberados[['GuidProduto','Data Comitê','Cliente','Deliberação','CETIP','ISIN']]

ativos_venda = pd.merge(right = carteira_atual,left=ativos_deliberados,on='GuidProduto',how='left')
ativos_venda = ativos_venda[ativos_venda['GuidProduto'].notnull()]


#Retorna apenas os clientes que passaram pelo comite e as posições que foram deliberadas e precisam ser vendidas
string = carteira_atual['NomeContaCRM'].drop_duplicates().to_list()
substr = ativos_deliberados['Cliente'].drop_duplicates().to_list()

def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

carteiras = Filter(string, substr)


#Ativos que precisam ser vendiddos
ativos_venda_comite = ativos_venda.loc[ativos_venda['NomeContaCRM'].isin(carteiras)][['NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','CETIP','ISIN','QuantidadeFinal','FinanceiroFuturo']].drop_duplicates().sort_values(by='NomeContaCRM').reset_index(drop=True)
dados_po_cadastro = bawm.po_cadastro(apenas_fundos=False)[['GuidContaCRM','NomeContaCRM']]
dados_crm= crm.account()[['accountid','createdon']].rename(columns={'accountid':'GuidContaCRM'})
dados_crm['GuidContaCRM'] = dados_crm['GuidContaCRM'].str.lower()
ativos_venda_comite = pd.merge(left=ativos_venda_comite,right=dados_po_cadastro,on='NomeContaCRM',how='left')
ativos_venda_comite = pd.merge(left=ativos_venda_comite,right=dados_crm,on='GuidContaCRM',how='left')
ativos_venda_comite['Dias_Implementação']=hoje - pd.to_datetime(ativos_venda_comite['createdon'])
ativos_venda_comite['Dias_Implementação'] = ativos_venda_comite['Dias_Implementação'].apply(lambda x : int(str(x).split() [0]))
ativos_venda_comite = ativos_venda_comite.sort_values(by=['Dias_Implementação'],ascending=False).drop_duplicates().reset_index(drop=True).drop(columns=['GuidContaCRM','createdon'])
ativos_venda_comite['CETIP'] = ativos_venda_comite['CETIP'].str.upper()
ativos_venda_comite['ISIN'] = ativos_venda_comite['ISIN'].str.upper()

In [13]:
#Retorna os ativos que não são recomendados nas carteiras dos clientes do comitê que não passaram por deliberação

ativos_comite = list(ativos_deliberados[ativos_deliberados['GuidProduto'].notnull()]['GuidProduto'].drop_duplicates())
ativos_nao_deliberados = carteira_atual.loc[carteira_atual['NomeContaCRM'].isin(carteiras)]
ativos_nao_deliberados = ativos_nao_deliberados.loc[~ativos_nao_deliberados['GuidProduto'].isin(ativos_comite)]
ativos_nao_deliberados = pd.merge(left=ativos_nao_deliberados, right=produtos,on='GuidProduto',how='left')
ativos_nao_deliberados = ativos_nao_deliberados[(ativos_nao_deliberados['Rating']!='Monitorado')&(ativos_nao_deliberados['Rating']!='Recomendado')].reset_index(drop=True)
ativos_nao_deliberados = ativos_nao_deliberados[~ativos_nao_deliberados['NomeProduto'].str.contains('COE')]

#Bancos emissores aprovados
emissores_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
emissores_aprovados = list(emissores_aprovados['Banco'])
emissores = ativos_nao_deliberados['emissor']
emissores_aprovados = [emissor if emissor in emissores_aprovados else np.nan for emissor in emissores]
emissores_aprovados = [emissor for emissor in emissores_aprovados if str(emissor) != 'nan']

#Retirar bancos aprovados
ativos_nao_deliberados = ativos_nao_deliberados.loc[~ativos_nao_deliberados['emissor'].isin(emissores_aprovados)].reset_index(drop=True)

In [14]:
#Puxando os ativos bancários que não `possuem CETIP/ISIN

ativos_bancos = ['CDB','LCI','LCA','LIG']
ativos_bancos_venda = ativos_nao_deliberados.loc[ativos_nao_deliberados['TipoProduto'].isin(ativos_bancos)].rename(columns={'emissor':'NomeEmissor'})
ativos_bancos_venda['Deliberação']= 'Não rastreado'
ativos_bancos_venda['Data Comitê']= 'Não rastreado'
ativos_bancos_venda['Cliente']= ativos_bancos_venda['NomeContaCRM']
ativos_bancos_venda= ativos_bancos_venda[['GuidProduto', 'Data Comitê', 'Cliente', 'Deliberação', 'CETIP', 'ISIN','DataArquivo', 'NomeContaCRM', 'TipoProduto', 'NomeProduto','NomeEmissor', 'QuantidadeFinal', 'FinanceiroFuturo']]
ativos_venda = pd.concat([ativos_venda,ativos_bancos_venda])
ativos_venda['Justificativa']=np.nan

In [15]:
#Retorna os ativos que foram deliberados para vendas mas estão nas carteoras de outros clientes
ativos_deliberados_outras_cart = carteira_atual.loc[~carteira_atual['NomeContaCRM'].isin(carteiras)]
ativos_deliberados_outras_cart = ativos_deliberados_outras_cart.loc[ativos_deliberados_outras_cart['GuidProduto'].isin(ativos_comite)]
ativos_deliberados_outras_cart = ativos_deliberados_outras_cart[~ativos_deliberados_outras_cart['NomeContaCRM'].str.contains('(Monitorado)')].drop_duplicates().sort_values(by=['NomeProduto','NomeContaCRM']).reset_index(drop=True)
ativos_deliberados_outras_cart['Justificativa']=np.nan

In [16]:
#Salvando o excel para 1LoD e enviando e-mail

#Gerando excel para e-mail:
with pd.ExcelWriter('C:/Temp/Foundation/Comite/FLoD.xlsx') as writer:  
    ativos_nao_deliberados.to_excel(writer, sheet_name='ativos_n_deliberados')
    verificar.to_excel(writer, sheet_name='verificar_deliberacao')
    produtos_aprovados.to_excel(writer, sheet_name='ativos_aprovados')
    ativos_não_localizados.to_excel(writer, sheet_name='ativos_n_localizados')

    

#Envio do email

subject='[FLds] - Comitê de Exceção-Ajustes des ativos'
to=['mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com','fernando.pavicic@juliusbaer.com']
anexo = 'C:/Temp/Foundation/Comite/Verificar_FLod.xlsx'
text = f'''Prezados,<br>

<br>
*Favor verificar as informações da planilha em anexo.<br>
<br>
1) Ativos não deliberados no comitê de exceção que estão nas carteiras dos clientes - Pode ser falha de cadastro também.<br>
2) Produtos aprovados pelo JBB com deliberações incorretas.<br>
3) Ativos deliberados sem cadastro no CRM (pode ser cadastro incorreto no CRM / informação incorreta na planilha).<br>

<br>
<br>
'''

email = Email(to = to , subject = subject, text= text,send = False, attachments = 'C:/Temp/Foundation/Comite/FLoD.xlsx')
    



In [17]:
#Salvando o excel para PM e enviando e-mail

#Gerando excel para e-mail:
with pd.ExcelWriter('C:/Temp/Foundation/Comite/Ativos_venda.xlsx') as writer:  
    ativos_venda_comite.to_excel(writer, sheet_name='vender_ativos_comite')
    ativos_deliberados_outras_cart.to_excel(writer, sheet_name='ativos_p_zerar_outrosclientes')

#Envio do email

subject='[FLds] - Comitê de Exceção-Venda de Ativos'
to=['mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com','fernando.pavicic@juliusbaer.com','portfolio@juliusbaer.com','igor.davanzo@juliusbaer.com','andre.lett@juliusbaer.com','felipe.dexheimer@juliusbaer.com','marcos.sposito@juliusbaer.com','joao.freitas@juliusbaer.com','bruno.caltabellota@juliusbaer.com','bernard.ledoux@juliusbaer.com','vito.ferreira@juliusbaer.com']
anexo = 'C:/Temp/Foundation/Comite/Verificar_FLod.xlsx'
text = f'''Prezados,<br>

<br>
*Seguem ativos para venda (deliberados via comitê de Exceção).<br>
**Peço nos retornar toda sexta com a justificativa para a não venda.<br>
***A seunfa aba refere-se a ativos que estão em outras carteiras (não passaram pelo comitê) mas estão com call de zerar.<br>

<br>
<br>
'''

email = Email(to = to , subject = subject, text= text,send = False, attachments = 'C:/Temp/Foundation/Comite/Ativos_venda.xlsx')